In [117]:
import requests
import random
import json
import re
import datetime as dt
from bs4 import BeautifulSoup
from urllib.parse import unquote
from headers import headers_list

In [7]:
channel_id = 'UC9x0AN7BWHpCDHSm9NiJFJQ'
base_url = f'https://www.youtube.com/channel/{channel_id}/about'
page = requests.get(base_url, headers=random.choice(headers_list))
page

<Response [200]>

In [101]:
soup = BeautifulSoup(page.content, 'html.parser')
json_text = str(soup.find_all('script')).split('var ytInitialData = ')[-1].split(';</script>')[0]
res = json.loads(json_text)
res

{'responseContext': {'serviceTrackingParams': [{'service': 'GFEEDBACK',
    'params': [{'key': 'route', 'value': 'channel.about'},
     {'key': 'is_casual', 'value': 'false'},
     {'key': 'is_owner', 'value': 'false'},
     {'key': 'is_monetization_enabled', 'value': 'true'},
     {'key': 'num_shelves', 'value': '5'},
     {'key': 'browse_id', 'value': 'UC9x0AN7BWHpCDHSm9NiJFJQ'},
     {'key': 'logged_in', 'value': '0'},
     {'key': 'country-type', 'value': 'B'},
     {'key': 'e',
      'value': '24214554,39321668,24002025,24218143,24210094,24212036,24199774,24120819,24082662,24002022,24166867,24219713,24080738,24194386,24165080,24036947,23940248,24211628,39321827,23934970,24223701,24187377,24085811,24199724,24189951,24185614,39321844,24191629,24227543,24163012,24152442,24187043,24167177,24223705,24219459,24202006,24001373,24209349,24215196,24219033,23983296,24222766,24027700,24135692,24213358,24166247,24199709,23986017,24217829,23966208,24187516,24034168,24211178,24181174,24198739,2

In [16]:
res.keys()

dict_keys(['responseContext', 'contents', 'header', 'metadata', 'trackingParams', 'topbar', 'microformat'])

In [116]:
content = None

for con in res['contents']['twoColumnBrowseResultsRenderer']['tabs']:
    if 'tabRenderer' not in con:
        continue
    if 'content' in con['tabRenderer']:
        content = con['tabRenderer']['content']['sectionListRenderer']['contents'][0]['itemSectionRenderer']\
            ['contents'][0]['channelAboutFullMetadataRenderer']

In [141]:
def get_simple_text(content, info):
    try:
        return content[info]['simpleText']
    except:
        return None

def get_view_count(content):
    try:
        view_count = content['viewCountText']['simpleText']
        view_count = view_count.split()[0].replace(',', '')
        return int(view_count)
    except:
        return None

def get_join_date(content):
    try:
        join_date = content['joinedDateText']['runs'][-1]['text']
        return dt.datetime.strptime(join_date, '%d %b %Y').strftime('%Y-%m-%d')
    except:
        return None

def get_links(content):
    links = {}
    for con in content['primaryLinks']:
        try:
            title = con['title']['simpleText']
            url = con['navigationEndpoint']['urlEndpoint']['url']
            url = unquote(url).split('q=')[-1]
            links[title] = url
        except:
            pass
    return links

In [142]:
output = {
    'id': content['channelId'],
    'title': get_simple_text(content, 'title'),
    'description': get_simple_text(content, 'description'),
    'country': get_simple_text(content, 'country'),
    'url': content['canonicalChannelUrl'],
    'join_date': get_join_date(content),
    'view_count': get_view_count(content),
    'links': get_links(content)
}
output

{'id': 'UC9x0AN7BWHpCDHSm9NiJFJQ',
 'title': 'NetworkChuck',
 'description': "Welcome to NetworkChuck!\n\nI LOVE Information Technology!! My goal is to help as MANY PEOPLE AS POSSIBLE jump into a career in the IT field through obtaining IT Certifications. I talk about Cisco Certifications, CompTIA, AWS, Microsoft...pretty much EVERYTHING in IT. As a CBT Nuggets Trainer, it's also my passion to TEACH information technology and to make it FUN. \n\nIf your goals are to obtain your CCNA, CompTIA A+, CompTIA Network+...I'm here to help you with that goal in any way I can!\n\n*****Interested in Sponsoring NetworkChuck? Email chuck@networkchuck.com",
 'country': 'United States',
 'url': 'http://www.youtube.com/c/NetworkChuck',
 'join_date': '2014-04-27',
 'view_count': 99244135,
 'links': {'NetworkChuck.com': 'https://networkchuck.com',
  'Twitter': 'https://twitter.com/networkchuck',
  'Instagram': 'https://instagram.com/networkchuck',
  'Twitch': 'https://twitch.tv/networkchuck'}}

In [143]:
def get_channel_info(channel_id):
    base_url = f'https://www.youtube.com/channel/{channel_id}/about'
    page = requests.get(base_url, headers=random.choice(headers_list))
    if page.status_code != 200:
        print(page, page.reason)
        return
    soup = BeautifulSoup(page.content, 'html.parser')
    json_text = str(soup.find_all('script')).split('var ytInitialData = ')[-1].split(';</script>')[0]
    res = json.loads(json_text)
    # Get content
    content = None
    for con in res['contents']['twoColumnBrowseResultsRenderer']['tabs']:
        if 'tabRenderer' not in con:
            continue
        if 'content' in con['tabRenderer']:
            content = con['tabRenderer']['content']['sectionListRenderer']['contents'][0]['itemSectionRenderer']\
                ['contents'][0]['channelAboutFullMetadataRenderer']
    if content is None:
        return
    return {
        'id': content['channelId'],
        'title': get_simple_text(content, 'title'),
        'description': get_simple_text(content, 'description'),
        'country': get_simple_text(content, 'country'),
        'url': content['canonicalChannelUrl'],
        'join_date': get_join_date(content),
        'view_count': get_view_count(content),
        'links': get_links(content)
    }

In [144]:
get_channel_info('UCBwmMxybNva6P_5VmxjzwqA')

{'id': 'UCBwmMxybNva6P_5VmxjzwqA',
 'title': 'Apna College',
 'description': 'Hey guys, welcome to this new channel - Apna College. Here Shradha Didi and I(your Aman Bhaiya) will help you in finding your right college, career options, soft skills and will also help you learn to code. \nChalo Phodte hain!\nFeel free to contact Shradha Didi for Seminars, Hackathons & Collaborations at the given email id below.\n\n\n',
 'country': None,
 'url': 'http://www.youtube.com/c/ApnaCollegeOfficial',
 'join_date': None,
 'view_count': 170936519,
 'links': {'Facebook': 'https://www.facebook.com/amandhattarwal',
  'Twitter': 'https://www.twitter.com/amandhattarwal',
  'Instagram': 'https://www.instagram.com/dhattarwalaman'}}

In [145]:
get_channel_info('UCLpovxJVLBZrXJGCymB6LYw')

{'id': 'UCLpovxJVLBZrXJGCymB6LYw',
 'title': 'Namanh Kapur',
 'description': "Hi! 👋 Welcome to my channel — I'm a 23 year old, recent graduate working at a high growth startup! Follow along for insights into the tech industry, all things startup, and what to expect as a recent-grad remote software engineer. I also dabble in sketch-comedy and investing. Thanks for being here!\n \nMilestones:\n[2022]\nJan 05 -- 0 subs (Video #1)\nFeb 07 -- 100 subs\nFeb 25 -- 500 subs\nMar 22 -- 1,000 subs\nMar 30 -- 5,000 subs\nApr 01 -- 10,000 subs\nApr 23 -- 25,000 subs\n",
 'country': 'United States',
 'url': 'http://www.youtube.com/c/NamanhKapur',
 'join_date': None,
 'view_count': 2132793,
 'links': {'namanhkapur.com': 'namanhkapur.com'}}

In [146]:
get_channel_info('UCsvqVGtbbyHaMoevxPAq9Fg')

{'id': 'UCsvqVGtbbyHaMoevxPAq9Fg',
 'title': 'Simplilearn',
 'description': 'Simplilearn is the world’s #1 online Bootcamp and one of the world’s leading certification training providers. Based in San Francisco, California, and Bangalore, India, we provide training in areas where technologies and best practices are changing rapidly, and the demand for qualified candidates significantly exceeds the supply. We have trained over 3,000,000 professionals, have over 2000 qualified trainers on board, and offer over 400 courses with 40 plus global accreditations. With live instructions from leading experts, interactive labs & projects, peer-to-peer collaboration, on-demand lessons, and 24/7 learning support, we provide learners with a comprehensive curriculum at a fraction of the cost of an on-campus program. \n\nFor more information, visit https://www.simplilearn.com/\nOur FREE courses via SkillUp by Simplilearn now come with Completion Certificates! :https://www.simplilearn.com/skillup-free-